## Imbalance class

In [ ]:
# Importing necessary libraries
import numpy as np
import pandas as pd

In [ ]:
# https://www.kaggle.com/datasets/uciml/breast-cancer-wisconsin-data?resource=download

df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data', header=None)

In [ ]:
# selects all rows and columns from the third column (indicating the features) and converting to a numpy array.
# selects all rows of the second column (ndicating the target variable) and converting to a numpy array

X = df.loc[:, 2:].values
y = df.loc[:, 1].values

# X matrix has dimensions (n_samples, n_features) and y is a 1-dimensional vector of length n_samples.

In [ ]:
# select the first five rows of the 'X' array

X[:5]

array([[1.799e+01, 1.038e+01, 1.228e+02, 1.001e+03, 1.184e-01, 2.776e-01,
        3.001e-01, 1.471e-01, 2.419e-01, 7.871e-02, 1.095e+00, 9.053e-01,
        8.589e+00, 1.534e+02, 6.399e-03, 4.904e-02, 5.373e-02, 1.587e-02,
        3.003e-02, 6.193e-03, 2.538e+01, 1.733e+01, 1.846e+02, 2.019e+03,
        1.622e-01, 6.656e-01, 7.119e-01, 2.654e-01, 4.601e-01, 1.189e-01],
       [2.057e+01, 1.777e+01, 1.329e+02, 1.326e+03, 8.474e-02, 7.864e-02,
        8.690e-02, 7.017e-02, 1.812e-01, 5.667e-02, 5.435e-01, 7.339e-01,
        3.398e+00, 7.408e+01, 5.225e-03, 1.308e-02, 1.860e-02, 1.340e-02,
        1.389e-02, 3.532e-03, 2.499e+01, 2.341e+01, 1.588e+02, 1.956e+03,
        1.238e-01, 1.866e-01, 2.416e-01, 1.860e-01, 2.750e-01, 8.902e-02],
       [1.969e+01, 2.125e+01, 1.300e+02, 1.203e+03, 1.096e-01, 1.599e-01,
        1.974e-01, 1.279e-01, 2.069e-01, 5.999e-02, 7.456e-01, 7.869e-01,
        4.585e+00, 9.403e+01, 6.150e-03, 4.006e-02, 3.832e-02, 2.058e-02,
        2.250e-02, 4.571e-03, 2.357e

In [ ]:
# slice the first five elements from the y array

y[:5]

array(['M', 'M', 'M', 'M', 'M'], dtype=object)

In [ ]:
'''
Importing the LabelEncoder class from the sklearn.preprocessing module
Creating an instance of the LabelEncoder class and fitting it to the target variable y
Printing the unique classes in the target variable
'''
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)
le.classes_

array(['B', 'M'], dtype=object)

- Creating a new array X_imb by stacking vertically X elements where y equals 0 and X elements where y equals 1 up to the 40th element.
- This results in a new X_imb array that is imbalanced, with many more examples where y is 0 than where y is 1.
- Creating a new array y_imb by concatenating horizontally y elements where y equals 0 and y elements where y equals 1 up to the 40th element.
- This results in a new y_imb array that is imbalanced, with many more examples where y is 0 than where y is 1.

In [ ]:
X_imb = np.vstack((X[y == 0], X[y == 1][:40]))
y_imb = np.hstack((y[y == 0], y[y == 1][:40]))

- originally consisted of 357 benign tumors (class 0) and 212 malignant tumors (class 1)
- took all 357 benign tumor samples and stacked them with the first 40 malignant samples to create a stark class imbalance
-  to compute the accuracy of a model that always predicts the majority class (benign, class 0), we would achieve a prediction accuracy of approximately 90 percent:

In [ ]:
# creates a numpy array with zeros of shape y_imb.shape[0] and assigns it to y_pred.
y_pred = np.zeros(y_imb.shape[0])

# computes the mean accuracy of y_pred compared to y_imb. It multiplies the boolean array resulting from the comparison by 100 and returns the result as a percentage.
np.mean(y_pred == y_imb) * 100

89.92443324937027

In [ ]:
# resampling data involves randomly drawing samples from a dataset with replacement to create a new dataset with a different distribution of values.
from sklearn.utils import resample

In [ ]:
print('Number of class 1 samples before:', X_imb[y_imb == 1].shape[0])

Number of class 1 samples before: 40


- dealing with highly unbalanced datasets is called resampling
- consists of
    - removing samples from the majority class (under-sampling)
        - randomly removing observations from the majority class to prevent its signal from dominating the learning algorithm. The most common heuristic for doing so is resampling without replacement
        - First, separate observations from each class into different datasets.
        - Next, resample the majority class without replacement, setting the number of samples to match that of the minority class.
        - Finally, combine the down-sampled majority class dataset with the original minority class dataset.
    - adding more examples from the minority class (over-sampling or up-sampling) - can cause overfitting
        - First separate observations from each class into different datasets.
        - Next, resample the minority class with replacement, setting the number of samples to match that of the majority class.
        - Finally, combine the up-sampled minority class dataset with the original majority class dataset.

![image.png](attachment:image.png)

## Oversampling

In [ ]:
#  resamples the minority class (y_imb == 1) with replacement to have the same number of samples as the majority class (y_imb == 0).

X_upsampled, y_upsampled = resample(X_imb[y_imb == 1], y_imb[y_imb == 1],
                                    replace=True, #sample with replacement
                                    n_samples=X_imb[y_imb == 0].shape[0], # to match majority class
                                    random_state=123)

In [ ]:
# prints the number of samples after upsampling the minority class.

print('Number of class 1 samples after:', X_upsampled.shape[0])

In [ ]:
# concatenate the upsampled minority class with the original majority class.

X_bal = np.vstack((X[y == 0], X_upsampled))
y_bal = np.hstack((y[y == 0], y_upsampled))

In [ ]:
# initializes y_pred to an array of zeros with the length equal to the number of samples in the concatenated dataset (y_bal).
# Then it calculates the percentage of correct predictions by comparing y_pred and y_bal.
y_pred = np.zeros(y_bal.shape[0])
np.mean(y_pred == y_bal) * 100

## Undersampling

In [ ]:
# This code initializes an array of zeros named 'y_pred' with the same shape as the input array 'y_imb'.
# The mean of the comparison between 'y_pred' and 'y_imb' is calculated and multiplied by 100. This gives the percentage of times 'y_pred' is equal to 'y_imb'.

y_pred = np.zeros(y_imb.shape[0])
np.mean(y_pred == y_imb) * 100

In [ ]:
X_downsampled, y_downsampled = resample(X_imb[y_imb == 0], y_imb[y_imb == 0],
                                    replace=False, #sample without replacement
                                    n_samples=X_imb[y_imb == 1].shape[0], # to match minority class
                                    random_state=123)

In [ ]:
print('Number of class 0 samples after:', X_downsampled.shape[0])

In [ ]:
X_bal2 = np.vstack((X[y == 1][:40], X_downsampled))
y_bal2 = np.hstack((y[y == 1][:40], y_downsampled))

In [ ]:
y_bal2

In [ ]:
y_pred = np.zeros(y_bal2.shape[0])
np.mean(y_pred == y_bal2) * 100

# Exercise

In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Load the breast cancer wisconsin dataset
data = load_breast_cancer()

# Split the data into features and labels
X, y = data.data, data.target

# Create an imbalanced dataset by randomly sampling 100 instances from the majority class (label 0) and keeping all instances from the minority class (label 1)
X_imb = np.vstack((X[y == 0][:100], X[y == 1]))
y_imb = np.hstack((y[y == 0][:100], y[y == 1]))

# Upsample the minority class to match the number of instances in the majority class
X_bal, y_bal = resample(X_imb[y_imb == 1], y_imb[y_imb == 1], replace=True, n_samples=X_imb[y_imb == 0].shape[0], random_state=123)

# Split the data into training and testing sets using a 70/30 split
X_train, X_test, y_train, y_test = train_test_split(X_bal, y_bal, test_size=0.3, random_state=123)

# Train a logistic regression model on the balanced data and test it on the testing set
clf_bal = LogisticRegression(random_state=123)
clf_bal.fit(X_train, y_train)
y_pred_bal = clf_bal.predict(X_test)

# Compare the results to a logistic regression model trained on the imbalanced data and tested on the same testing set
clf_imb = LogisticRegression(random_state=123)
clf_imb.fit(X_imb, y_imb)
y_pred_imb = clf_imb.predict(X_test)

# Print the accuracy scores
print("Accuracy score (balanced data):", accuracy_score(y_test, y_pred_bal))
print("Accuracy score (imbalanced data):", accuracy_score(y_test, y_pred_imb))
